In [177]:
import pandas as pd
import numpy as np

#load in CPS data
#read text file separted by commas
df=pd.read_csv("CPS2014extract.csv")
df.head(5)

#view list of variable names
columnsNamesArr = df.columns.values
columnsNamesArr

array(['h_seq', 'h_year', 'h_hhtype', 'h_numper', 'h_month', 'h_mis',
       'h_hhnum', 'gestcen', 'gestfips', 'ffpos', 'fpersons', 'pppos',
       'a_lineno', 'a_age', 'a_sex', 'a_hga', 'prdtrace', 'p_stat',
       'pehspnon', 'marsupwt', 'a_clswkr', 'prerelg', 'a_uslhrs',
       'a_hrlywk', 'a_lfsr', 'a_mjind', 'a_mjocc', 'prcow1', 'pemlr',
       'workyn', 'wkswork', 'phmemprs', 'hrswk', 'clwk', 'weclw',
       'wemocg', 'wemind', 'ljcw', 'noemp', 'pearnval', 'hiemp',
       'i_noemp', 'pextra1'], dtype=object)

In [178]:
#Subset of Test and Training Varaibles + Weight
df2 = df[['a_hrlywk', 'a_age', 'a_sex', 'a_hga', 'prdtrace','a_mjind', 'a_mjocc', 'pehspnon', 'prerelg', 'marsupwt']]

#Review predictors: gender, race, age, age squared, education attainment, occupation, and industry.
#print frequencies
for i in df2:
    print(df2[i].value_counts()) #sex 1=, 2=

d = df2.copy()

0    55368
1     7261
2     5425
Name: a_hrlywk, dtype: int64
33    1666
43    1641
35    1619
42    1614
46    1612
45    1611
44    1591
50    1582
34    1576
48    1568
41    1567
32    1549
40    1542
49    1530
47    1529
38    1523
39    1521
52    1507
31    1499
51    1490
36    1490
37    1473
30    1454
28    1443
53    1411
29    1387
54    1387
56    1355
55    1343
26    1324
      ... 
22    1100
60    1074
59    1069
20    1039
19     919
61     905
62     885
18     828
63     761
64     658
17     585
65     578
66     525
67     442
16     367
68     296
69     261
70     229
72     205
71     192
80     178
15     163
73     140
74     123
75     105
77      97
85      91
76      81
78      70
79      70
Name: a_age, Length: 67, dtype: int64
1    35418
2    32636
Name: a_sex, dtype: int64
39    18183
43    14378
40    12836
44     5920
42     3946
41     3161
37     1715
46     1285
36     1279
45     1120
35     1020
33      922
38      890
34      857
32      401
3

In [179]:
#Outcome variable = Paid by the hour on this job? Values: 0 = not earning eligible; 1 = earnings eligible
#prerelg = earning flag (0= not in universe, 1=Yes, 2=No)
d['a_hrlywk'].value_counts() 
d['paid_hrly'] = np.where((d['prerelg']==1), 0, np.nan)
#print (pd.crosstab(d.paid_hrly, d.prerelg, dropna=False)) #check that it worked

d['paid_hrly'] = np.where(((d['prerelg']==1) & (d['a_hrlywk']==1)),1, d['paid_hrly'])
print (pd.crosstab(d.paid_hrly, d.prerelg, dropna=False)) #check that it worked


prerelg    0     1
paid_hrly         
0.0        0  5425
1.0        0  7261


In [180]:
#generate Age^2
d['a_age_sq'] = d['a_age']**2
d[['a_age_sq', 'a_age']].head(5) #check that it worked

,a_age_sq,a_age
0,5329,73
1,5184,72
2,1156,34
3,1681,41
4,256,16


In [181]:
#generate race dummies (Values:1 -26, #25 missing in data
# print(d['prdtrace'].nunique()) #25 categories 
print(d['prdtrace'].value_counts(sort=True))  #race)
print(d['pehspnon'].value_counts(sort=True))  #hispanic)

#black (non-hispanic)
d['black'] = np.where((d['prdtrace']==2) & (d['pehspnon'] == 2), 1, 0)
#print (pd.crosstab(d.black, d.prdtrace, margins=True)) #check that it worked

#asian (non-hispanic)
d['asian'] = np.where((d['prdtrace']==4) & (d['pehspnon'] == 2), 1, 0)
#print (pd.crosstab(d.asian, d.prdtrace, margins=True)) #check that it worked

#hispanic
d['hisp'] = np.where((d['pehspnon'] == 1), 1, 0)
#print (pd.crosstab(d.hisp, d.pehspnon, margins=True)) #check that it worked

#other (American Indian, hawaiian/pacific islander - non-hispanic)
d['other'] = np.where((d['prdtrace']==3) | ((d['prdtrace']>=5) & (d['prdtrace'] <= 26)) & (d['pehspnon'] == 2), 1, 0)
pd.crosstab(d.other, d.prdtrace, margins=True) #check that it worked

#white (non-hispanic) - LEAVE OUT
d['white'] = np.where((d['prdtrace']==1) & (d['pehspnon'] == 2), 1, 0)
pd.crosstab(d.white, d.prdtrace, margins=True) #check that it worked

1     54683
2      7063
4      3930
3       752
7       440
5       344
6       271
8       221
10       69
15       68
9        61
16       54
21       53
11       10
12        7
19        5
17        4
14        4
26        3
13        3
25        2
18        2
20        2
23        2
22        1
Name: prdtrace, dtype: int64
2    54877
1    13177
Name: pehspnon, dtype: int64


prdtrace,1,2,3,4,5,6,7,8,9,10,...,17,18,19,20,21,22,23,25,26,All
white,,,,,,,,,,,,,,,,,,,,,
0,12249,7063,752,3930,344,271,440,221,61,69,...,4,2,5,2,53,1,2,2,3,25620
1,42434,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,42434
All,54683,7063,752,3930,344,271,440,221,61,69,...,4,2,5,2,53,1,2,2,3,68054


In [182]:
#Educational Attainment Dummies (values: 0 = children, 31-46)
print(d['a_hga'].nunique()) #15 categories
#print(d['a_hga'].value_counts(sort=True))  #edu attain

#Less than HS diploma
d['ltHS'] = np.where((d['a_hga']<=38), 1, 0)
print (pd.crosstab(d.ltHS, d.a_hga, margins=True)) #check that it worked

#Some college
d['someCol'] = np.where((d['a_hga']>= 40) & (d['a_hga']<= 42), 1, 0)
print (pd.crosstab(d.someCol, d.a_hga, margins=True)) #check that it worked

#BA
d['BA'] = np.where((d['a_hga']==43), 1, 0)
print (pd.crosstab(d.BA, d.a_hga, margins=True)) #check that it worked

#Grad School
d['GradSch'] = np.where((d['a_hga']>=44), 1, 0)
print (pd.crosstab(d.GradSch, d.a_hga, margins=True)) #check that it worked

#HS - LEAVE OUT
d['HS'] = np.where((d['a_hga']>=39), 1, 0)

16
a_hga   31   32   33   34    35    36    37   38     39     40    41    42  \
ltHS                                                                         
0        0    0    0    0     0     0     0    0  18183  12836  3161  3946   
1      141  401  922  857  1020  1279  1715  890      0      0     0     0   
All    141  401  922  857  1020  1279  1715  890  18183  12836  3161  3946   

a_hga     43    44    45    46    All  
ltHS                                   
0      14378  5920  1120  1285  60829  
1          0     0     0     0   7225  
All    14378  5920  1120  1285  68054  
a_hga     31   32   33   34    35    36    37   38     39     40    41    42  \
someCol                                                                        
0        141  401  922  857  1020  1279  1715  890  18183      0     0     0   
1          0    0    0    0     0     0     0    0      0  12836  3161  3946   
All      141  401  922  857  1020  1279  1715  890  18183  12836  3161  3946   

a_hga

In [183]:
#Gender dummies
print(d['a_sex'].nunique()) #15 categories
print(d['a_sex'].value_counts(sort=True))  #edu attain

#Less than HS diploma
d['male'] = np.where((d['a_sex']==1), 1, 0)
d['female'] = np.where((d['a_sex']==2), 1, 0)

print (pd.crosstab(d.male, d.a_sex, margins=True)) #check that it worked
print (pd.crosstab(d.female, d.a_sex, margins=True)) #check that it worked


2
1    35418
2    32636
Name: a_sex, dtype: int64
a_sex      1      2    All
male                      
0          0  32636  32636
1      35418      0  35418
All    35418  32636  68054
a_sex       1      2    All
female                     
0       35418      0  35418
1           0  32636  32636
All     35418  32636  68054


In [184]:
#Occupation (12 categories. Values: 0=not in universe/children, 1-11. 11=armed forces)
print(d['a_mjocc'].nunique()) #15 categories
#print(d['a_mjocc'].value_counts(sort=True))

d['occ_1'] = np.where((d['a_mjocc']== 1), 1, 0)
#print (pd.crosstab(d.occ_1, d.a_mjocc, margins=True)) #check that it worked
print(d['occ_1'].value_counts(dropna=False))  

d['occ_2'] = np.where((d['a_mjocc']== 2), 1, 0)
d['occ_3'] = np.where((d['a_mjocc']== 3), 1, 0)
d['occ_4'] = np.where((d['a_mjocc']== 4), 1, 0)
d['occ_5'] = np.where((d['a_mjocc']== 5), 1, 0)
d['occ_6'] = np.where((d['a_mjocc']== 6), 1, 0)
d['occ_7'] = np.where((d['a_mjocc']== 7), 1, 0)
d['occ_8'] = np.where((d['a_mjocc']== 8), 1, 0)
d['occ_9'] = np.where((d['a_mjocc']== 9), 1, 0)
d['occ_10'] = np.where((d['a_mjocc']== 10), 1, 0)

d['occ_1'] = np.where(((d['a_mjocc']==0)| (d['a_mjocc']==11)), np.nan, d['occ_1'])
d['occ_2'] = np.where(((d['a_mjocc']==0)| (d['a_mjocc']==11)), np.nan, d['occ_2'])
d['occ_3'] = np.where(((d['a_mjocc']==0)| (d['a_mjocc']==11)), np.nan, d['occ_3'])
d['occ_4'] = np.where(((d['a_mjocc']==0)| (d['a_mjocc']==11)), np.nan, d['occ_4'])
d['occ_5'] = np.where(((d['a_mjocc']==0)| (d['a_mjocc']==11)), np.nan, d['occ_5'])
d['occ_6'] = np.where(((d['a_mjocc']==0)| (d['a_mjocc']==11)), np.nan, d['occ_6'])
d['occ_7'] = np.where(((d['a_mjocc']==0)| (d['a_mjocc']==11)), np.nan, d['occ_7'])
d['occ_8'] = np.where(((d['a_mjocc']==0)| (d['a_mjocc']==11)), np.nan, d['occ_8'])
d['occ_9'] = np.where(((d['a_mjocc']==0)| (d['a_mjocc']==11)), np.nan, d['occ_9'])
d['occ_10'] = np.where(((d['a_mjocc']==0)| (d['a_mjocc']==11)), np.nan, d['occ_10'])
print(d['occ_1'].value_counts(dropna=False))  



12
0    57947
1    10107
Name: occ_1, dtype: int64
 0.0    54358
 1.0    10107
NaN      3589
Name: occ_1, dtype: int64


In [185]:
#Industry (15 categories. Values: 0=not in universe/children, 1-14. 14=armed forces)
print(d['a_mjind'].nunique()) #15 categories
#print(d['a_mjind'].value_counts(sort=True)) 

d['ind_1'] = np.where((d['a_mjind']== 1), 1, 0)
#print (pd.crosstab(d.ind_1, d.a_mjind, margins=True)) #check that it worked
print(d['ind_1'].value_counts(dropna=False))  

d['ind_2'] = np.where((d['a_mjind']== 2), 1, 0)
d['ind_3'] = np.where((d['a_mjind']== 3), 1, 0)
d['ind_4'] = np.where((d['a_mjind']== 4), 1, 0)
d['ind_5'] = np.where((d['a_mjind']== 5), 1, 0)
d['ind_6'] = np.where((d['a_mjind']== 6), 1, 0)
d['ind_7'] = np.where((d['a_mjind']== 7), 1, 0)
d['ind_8'] = np.where((d['a_mjind']== 8), 1, 0)
d['ind_9'] = np.where((d['a_mjind']== 9), 1, 0)
d['ind_10'] = np.where((d['a_mjind']== 10), 1, 0)
d['ind_11'] = np.where((d['a_mjind']== 11), 1, 0)
d['ind_12'] = np.where((d['a_mjind']== 12), 1, 0)
d['ind_13'] = np.where((d['a_mjind']== 13), 1, 0)

d['ind_1'] = np.where(((d['a_mjind']==0)| (d['a_mjind']==14)), np.nan, d['ind_1'])
d['ind_2'] = np.where(((d['a_mjind']==0)| (d['a_mjind']==14)), np.nan, d['ind_2'])
d['ind_3'] = np.where(((d['a_mjind']==0)| (d['a_mjind']==14)), np.nan, d['ind_3'])
d['ind_4'] = np.where(((d['a_mjind']==0)| (d['a_mjind']==14)), np.nan, d['ind_4'])
d['ind_5'] = np.where(((d['a_mjind']==0)| (d['a_mjind']==14)), np.nan, d['ind_5'])
d['ind_6'] = np.where(((d['a_mjind']==0)| (d['a_mjind']==14)), np.nan, d['ind_6'])
d['ind_7'] = np.where(((d['a_mjind']==0)| (d['a_mjind']==14)), np.nan, d['ind_7'])
d['ind_8'] = np.where(((d['a_mjind']==0)| (d['a_mjind']==14)), np.nan, d['ind_8'])
d['ind_9'] = np.where(((d['a_mjind']==0)| (d['a_mjind']==14)), np.nan, d['ind_9'])
d['ind_10'] = np.where(((d['a_mjind']==0)| (d['a_mjind']==14)), np.nan, d['ind_10'])
d['ind_11'] = np.where(((d['a_mjind']==0)| (d['a_mjind']==14)), np.nan, d['ind_11'])
d['ind_12'] = np.where(((d['a_mjind']==0)| (d['a_mjind']==14)), np.nan, d['ind_12'])
d['ind_13'] = np.where(((d['a_mjind']==0)| (d['a_mjind']==14)), np.nan, d['ind_13'])
print(d['ind_1'].value_counts(dropna=False))  





15
0    66879
1     1175
Name: ind_1, dtype: int64
 0.0    63290
NaN      3589
 1.0     1175
Name: ind_1, dtype: int64


In [190]:
#Drop values where Outcome Variable is NaN
#d.dropna(subset=['paid_hrly'])
#d = d[np.isfinite(d['paid_hrly'])]

# Save data
d.to_csv("cps_clean_2014.csv", index=False, header=True)

In [191]:
d.count()

a_hrlywk     12686
a_age        12686
a_sex        12686
a_hga        12686
prdtrace     12686
a_mjind      12686
a_mjocc      12686
pehspnon     12686
prerelg      12686
marsupwt     12686
paid_hrly    12686
a_age_sq     12686
black        12686
asian        12686
hisp         12686
other        12686
white        12686
ltHS         12686
someCol      12686
BA           12686
GradSch      12686
HS           12686
male         12686
female       12686
occ_1        12686
occ_2        12686
occ_3        12686
occ_4        12686
occ_5        12686
occ_6        12686
occ_7        12686
occ_8        12686
occ_9        12686
occ_10       12686
ind_1        12686
ind_2        12686
ind_3        12686
ind_4        12686
ind_5        12686
ind_6        12686
ind_7        12686
ind_8        12686
ind_9        12686
ind_10       12686
ind_11       12686
ind_12       12686
ind_13       12686
dtype: int64